In [7]:
import sqlite3
import pandas as pd

DATABASE_PATH = 'data/World_Economies.db'
TABLE_NAME = 'Countries_by_GDP_etl_processed'

# Connect to the database
conn = sqlite3.connect(DATABASE_PATH)

In [8]:
# DataFrame of countries with GDP of 100(Billion) or more
with conn:
    cur = conn.cursor()
    
    query = f"SELECT * FROM {TABLE_NAME} WHERE GDP_USD_billion >= 100"
    df = pd.read_sql_query(query, conn)

    display(df)

,Country,GDP_USD_billion,year,region
0,United States,30337.16,2025,North America
1,China,19534.89,2025,East Asia
2,Germany,4921.56,2025,Western Europe
3,Japan,4389.33,2025,East Asia
4,India,4271.92,2025,South Asia
...,...,...,...,...
67,Kenya,116.32,2024,Sub-Saharan Africa
68,Angola,113.29,2024,Sub-Saharan Africa
69,Guatemala,112.37,2024,Latin America
70,Oman,110.99,2024,Middle East


In [9]:
# Mean GDP of top 5 countries by GDP in each region (sorted in descending order)
with conn:
    cur = conn.cursor()
    
    query = f"""
        SELECT region, ROUND(AVG(GDP_USD_billion), 2) as avg_gdp
        FROM (
            SELECT region, GDP_USD_billion,
                ROW_NUMBER() OVER (PARTITION BY region ORDER BY GDP_USD_billion DESC) AS rank
            FROM {TABLE_NAME}
        )
        WHERE rank <= 5
        GROUP BY region
        ORDER BY avg_gdp DESC

    """
    df = pd.read_sql_query(query, conn)

    display(df)

,region,avg_gdp
0,North America,8622.34
1,East Asia,5421.57
2,Western Europe,3244.49
3,Eastern Europe,1085.00
4,South Asia,1044.18
5,Latin America,797.57
6,Southeast Asia,711.89
7,Middle East,598.13
8,Oceania,436.66
9,Sub-Saharan Africa,215.48
